In [1]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm

In [10]:
map_name_mapper = {'ifn': 'inferno',
                   'ovp': 'overpass',
                   'mrg': 'mirage',
                   'nuk': 'nuke',
                   'd2': 'dust 2'}

In [11]:
txt_cols = ['result', 'map', 'score', 'pct_win', 'CT', 'game_id']
df = pd.read_csv('../data/jogos.txt', sep='\t', header=None)
df[11].fillna(df[9], inplace=True)
df.dropna(axis=1, inplace=True) # remove empty cols
df.columns = txt_cols
df['map'] = df['map'].map(map_name_mapper) # rename map names
df['game_id'] = df['game_id'].astype(int)
df['game_url'] = df['game_id'].apply(lambda x: f'https://gamersclub.com.br/lobby/match/{x}')
df['team_score'] = df['score'].apply(lambda x: x.split('x')[0]).astype(int)
df['foe_score'] = df['score'].apply(lambda x: x.split('x')[1]).astype(int)
df['score_diff'] = df['team_score'] - df['foe_score']
df.drop_duplicates('game_id', inplace=True)
df2 = df.copy()
df.shape

(83, 10)

In [4]:
options = webdriver.EdgeOptions()
options.add_argument("--headless")
navegador = webdriver.Edge(executable_path='../webdriver/msedgedriver.exe', options=options)
urls = df['game_url'].tolist()
games_data = pd.DataFrame(columns=['NAME', 'K', 'A', 'D', 'DIFF', 'ADR', 'KDR', 'KAST', 
                                    'S', 'T', 'FA', '1VSX', 'MK', 'FK', 'RP', 'GAME_ID'])
for url in tqdm(urls):
    navegador.get(url)
    myElem = WebDriverWait(navegador, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[9]/div/div/section/div[3]/div/div[2]')))
    match_info = navegador.find_elements(By.XPATH, '/html/body/div[2]/div[9]/div/div/section/div[3]/div/div[2]')
    match_data = []
    for info in match_info:
        info_ = info.text.split('\n')
        date = info_[1]
        duration = info_[3]
        id = int(info_[5])
        map = info_[7]
        match_data.extend([date, id, duration, map])

    cols = ['NAME', 'K', 'A', 'D', 'DIFF', 'ADR', 'KDR', 'KAST', 
        'S', 'T', 'FA', '1VSX', 'MK', 'FK', 'RP']
    match_table = navegador.find_elements(By.CLASS_NAME, 'tableMatch')
    start = 0
    table_data = []
    for players_stats in match_table:
        for tr in players_stats.find_elements(By.CLASS_NAME, 'tableMatch__container'):
            tds = tr.find_elements(By.TAG_NAME, 'td')
            output_lst = [td.text for td in tds]
            output_lst = list(filter(None, output_lst))
            output_lst = [output_lst[i:i+15] for i in range(0, len(output_lst), 15)]
            table_data.extend(output_lst)

    stats_df = pd.DataFrame(table_data, columns=cols)
    stats_df['GAME_ID'] = match_data[1]
    stats_df['GAME_DATE'] = match_data[0]
    stats_df['NAME'] = stats_df['NAME'].apply(lambda x: x.split('\n')[0])
    games_data = pd.concat([games_data, stats_df], ignore_index=True)

games_data['KAST'] = round(games_data['KAST'].str.replace('%', '').astype(int)/100, 2)


C:\Users\Carlos\AppData\Local\Temp\ipykernel_14152\2201716512.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Edge(executable_path='../webdriver/msedgedriver.exe', options=options)
100%|██████████| 83/83 [08:44<00:00,  6.32s/it]


In [12]:
games_data['GAME_ID'] = games_data['GAME_ID'].astype(int)
df3 = df2.merge(games_data[['GAME_ID', 'GAME_DATE']], left_on='game_id', right_on='GAME_ID')
df3.drop('GAME_ID', axis=1, inplace=True)
df3.drop_duplicates(inplace=True)

In [13]:
df3.shape

(83, 11)

In [15]:
df3['map'].unique().tolist()

['inferno', 'overpass', 'mirage', 'nuke', 'dust 2']

In [16]:
games_data.to_csv('../data/data_scrapped.csv')
df3.to_csv('../data/games_clean.csv')